<a href="https://colab.research.google.com/github/ma2sevich222/raw_stock_data_researches/blob/main/kalman%20smoother%20and%20RBM%20hidden%20rep%20clustering%20for%20stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import io
import plotly.express as px
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
import math
import plotly.graph_objects as go

In [ ]:
uploaded = files.upload()

Saving GC_2019_2022_30min.csv to GC_2019_2022_30min.csv


In [ ]:
df=pd.read_csv(io.BytesIO(uploaded['GC_2019_2022_30min.csv']) )

In [ ]:
class Kalman_Smoother():
   def __init__(self, data_frame, r =0.99, dNoise=2.7,en=None):
     self.Data = data_frame
     self.r = r
     self.dNoise = dNoise
     self.en = en
   def get_smoothed_df(self):
     colums_to_be_smoothed = ["Open","High","Low","Close"]
     smoothed_df = pd.DataFrame()
     smoothed_df['Datetime'] = self.Data['Datetime'].values
     for col in colums_to_be_smoothed:
       N = len(self.Data[col])
       smoo_ed_col = np.zeros(N)
       P = np.zeros(N)
       smoo_ed_col[0] = self.Data.loc[0, col]
       P[0] = 0
       for i in range(1,N):
         Pe = self.r*self.r*P[i-1] + self.en * self.en
         P[i] = (Pe * self.dNoise)/ (Pe+ self.dNoise)
         smoo_ed_col[i] = self.r*smoo_ed_col[i-1] + P[i]/self.dNoise*(self.Data.loc[i,col] - self.r*smoo_ed_col[i-1])
       smoothed_df[col] = smoo_ed_col
       smoothed_df["Volume"] = self.Data["Volume"].values
       

     return smoothed_df

In [ ]:
#real_close = df["Close"].values
smoother = Kalman_Smoother(df,en=0.8)
sm_df = smoother.get_smoothed_df()

In [ ]:
smoother.en

0.8

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(len(df))], y=df["Close"],
                    mode='lines',
                    name='close_real'))
fig.add_trace(go.Scatter(x=[i for i in range(len(sm_df))], y=sm_df["Close"],
                    mode='lines',
                    name='close_smoothed'))
fig.show()

In [ ]:
df[-41235:]

,Datetime,Open,High,Low,Close,Volume
0,2019-01-02 01:30:00,1282.2,1282.3,1280.8,1281.5,3850.0
1,2019-01-02 02:00:00,1281.5,1282.4,1281.1,1281.9,2668.0
2,2019-01-02 02:30:00,1281.9,1283.8,1281.8,1283.2,2813.0
3,2019-01-02 03:00:00,1283.3,1284.8,1282.7,1284.8,2988.0
4,2019-01-02 03:30:00,1284.8,1285.4,1284.2,1285.4,2125.0
...,...,...,...,...,...,...
41230,2022-08-24 12:00:00,1757.2,1758.2,1754.8,1757.9,3813.0
41231,2022-08-24 12:30:00,1757.9,1759.9,1757.1,1758.8,4533.0
41232,2022-08-24 13:00:00,1758.6,1760.2,1755.7,1758.6,5265.0
41233,2022-08-24 13:30:00,1758.6,1759.2,1756.2,1756.3,3452.0


In [ ]:
start_forward_time = "2021-11-01 00:00:00"
end_test_time = "2021-12-01 00:00:00"
forward_index = df[df["Datetime"] == start_forward_time].index[0]
end_test_index = df[df["Datetime"] == end_test_time].index[0]
train_df = df[forward_index - 10000:forward_index]
test_df = df[forward_index: end_test_index]
#real_test_close = real_close [forward_index: end_test_index]

In [ ]:
patch = 2
scaler = MinMaxScaler()
#scaler = StandardScaler()
BATCH_SIZE = 10
VISIBLE_UNITS = 5*patch  
HIDDEN_UNITS = 55
CD_K = 2
EPOCHS = 80



CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

if CUDA:
    torch.cuda.set_device(CUDA_DEVICE)

 КЛАССЫ

In [ ]:
class Datagenerator():

  def __init__(self, data_frame, patch):

    self.data = data_frame
    self.patch = patch
    

  def get_pached_samples(self):
    train_arr = self.data[['Open','High','Low','Close','Volume']].to_numpy()
    pached_samples =  np.array([train_arr[i-self.patch:i] for i in range(len(train_arr)+1) if i - self.patch >= 0])
    return pached_samples
  
  def scaling_data_and_flatten(self, pached_samples, Scaller):
    scaled_and_transformed = []
    for sample in pached_samples:
      sample_normlzd = Scaller.fit_transform(sample)
      scaled_and_transformed.append(sample_normlzd.flatten())
    return np.array(scaled_and_transformed)

  def return_OHLV(self):
    #self.data[['Datetime','Open','High','Low','Close','Volume']][self.patch-1:]
    OHLV_data = self.data[['Datetime','Open','High','Low','Close','Volume']][self.patch-1:]
    OHLV_data = OHLV_data.reset_index(drop=True)
    return OHLV_data

  def get_labels(self):
    labels = self.data['Signal'].to_numpy()
    return labels[self.patch-1:]



In [ ]:
class RBM():

    def __init__(self, num_visible, num_hidden, k,  learning_rate=1e-3, momentum_coefficient=0.01, weight_decay=1e-4, use_cuda=True):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.k = k
        self.learning_rate = learning_rate
        self.momentum_coefficient = momentum_coefficient
        self.weight_decay = weight_decay
        self.use_cuda = use_cuda
        self.weights = torch.randn(num_visible, num_hidden) * 0.1
        self.visible_bias = torch.ones(num_visible) * 0.5
        self.hidden_bias = torch.zeros(num_hidden)
        self.weights_momentum = torch.zeros(num_visible, num_hidden)
        self.visible_bias_momentum = torch.zeros(num_visible)
        self.hidden_bias_momentum = torch.zeros(num_hidden)

        if self.use_cuda:
            self.weights = self.weights.cuda()
            self.visible_bias = self.visible_bias.cuda()
            self.hidden_bias = self.hidden_bias.cuda()

            self.weights_momentum = self.weights_momentum.cuda()
            self.visible_bias_momentum = self.visible_bias_momentum.cuda()
            self.hidden_bias_momentum = self.hidden_bias_momentum.cuda()

    def sample_hidden(self, visible_probabilities):
        hidden_activations = torch.matmul(visible_probabilities, self.weights) + self.hidden_bias
        hidden_probabilities = self._sigmoid(hidden_activations)
        return hidden_probabilities

    def sample_visible(self, hidden_probabilities):
        visible_activations = torch.matmul(hidden_probabilities, self.weights.t()) + self.visible_bias
        visible_probabilities = self._sigmoid(visible_activations)
        return visible_probabilities

    def contrastive_divergence(self, input_data):
        # Positive phase
        positive_hidden_probabilities = self.sample_hidden(input_data)
        positive_hidden_activations = (positive_hidden_probabilities >= self._random_probabilities(self.num_hidden)).float()
        positive_associations = torch.matmul(input_data.t(), positive_hidden_activations)

        # Negative phase
        hidden_activations = positive_hidden_activations

        for step in range(self.k):
            visible_probabilities = self.sample_visible(hidden_activations)
            hidden_probabilities = self.sample_hidden(visible_probabilities)
            hidden_activations = (hidden_probabilities >= self._random_probabilities(self.num_hidden)).float()

        negative_visible_probabilities = visible_probabilities
        negative_hidden_probabilities = hidden_probabilities

        negative_associations = torch.matmul(negative_visible_probabilities.t(), negative_hidden_probabilities)

        # Update parameters
        self.weights_momentum *= self.momentum_coefficient
        self.weights_momentum += (positive_associations - negative_associations)

        self.visible_bias_momentum *= self.momentum_coefficient
        self.visible_bias_momentum += torch.sum(input_data - negative_visible_probabilities, dim=0)

        self.hidden_bias_momentum *= self.momentum_coefficient
        self.hidden_bias_momentum += torch.sum(positive_hidden_probabilities - negative_hidden_probabilities, dim=0)

        batch_size = input_data.size(0)

        self.weights += self.weights_momentum * self.learning_rate / batch_size
        self.visible_bias += self.visible_bias_momentum * self.learning_rate / batch_size
        self.hidden_bias += self.hidden_bias_momentum * self.learning_rate / batch_size

        self.weights -= self.weights * self.weight_decay  # L2 weight decay

        # Compute reconstruction error
        error = torch.sum((input_data - negative_visible_probabilities)**2)

        return error

    def _sigmoid(self, x):
        return 1 / (1 + torch.exp(-x))


    def _random_probabilities(self, num):
        random_probabilities = torch.rand(num)

        if self.use_cuda:
            random_probabilities = random_probabilities.cuda()

        return random_probabilities


class RBMDataset(Dataset):
    def __init__(self, X):
        self.x_train = torch.tensor(X, dtype=torch.float32)


    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        return self.x_train[idx]

In [ ]:
def take_log(el):
  return math.log(el)**1/3

RUN

In [ ]:
loger=np.vectorize(take_log)


In [ ]:
train_data = Datagenerator(train_df, patch)

In [ ]:
Train_X = train_data.get_pached_samples()
Train_X = loger(Train_X)
_X = train_data.scaling_data_and_flatten(Train_X, scaler)

In [ ]:
Train_X[0]


array([[2.50598518, 2.50613001, 2.50560469, 2.50569532, 2.54397217],
       [2.50569532, 2.50574969, 2.50475155, 2.50482424, 2.79018254]])

In [ ]:
train_dataset = RBMDataset(_X)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
rbm = RBM(VISIBLE_UNITS, HIDDEN_UNITS, CD_K, use_cuda=CUDA)

for epoch in range(EPOCHS):
    epoch_error = 0.0

    for batch in train_dataloader:
        
       
        #batch = batch.view(len(batch), VISIBLE_UNITS)  # flatten input data
        
        if CUDA:
            batch = batch.cuda()

        batch_error = rbm.contrastive_divergence(batch)

        epoch_error += batch_error

    print('Epoch Error (epoch=%d): %.4f' % (epoch, epoch_error))

Epoch Error (epoch=0): 25935.9531
Epoch Error (epoch=1): 25670.0254
Epoch Error (epoch=2): 25600.8965
Epoch Error (epoch=3): 25388.0430
Epoch Error (epoch=4): 25221.8906
Epoch Error (epoch=5): 25178.9199
Epoch Error (epoch=6): 24731.2402
Epoch Error (epoch=7): 24380.0898
Epoch Error (epoch=8): 23646.4453
Epoch Error (epoch=9): 22924.6895
Epoch Error (epoch=10): 22234.0566
Epoch Error (epoch=11): 21355.2656
Epoch Error (epoch=12): 20602.5859
Epoch Error (epoch=13): 20001.8242
Epoch Error (epoch=14): 19713.6465
Epoch Error (epoch=15): 19416.8555
Epoch Error (epoch=16): 19349.8281
Epoch Error (epoch=17): 19071.2129
Epoch Error (epoch=18): 19038.2305
Epoch Error (epoch=19): 19113.2422
Epoch Error (epoch=20): 18922.3438
Epoch Error (epoch=21): 18710.2793
Epoch Error (epoch=22): 18841.7070
Epoch Error (epoch=23): 18671.3203
Epoch Error (epoch=24): 18996.9590
Epoch Error (epoch=25): 18802.7324
Epoch Error (epoch=26): 18828.1973
Epoch Error (epoch=27): 19026.7676
Epoch Error (epoch=28): 18906.

In [ ]:
test_data = Datagenerator(test_df, patch)
test_X = test_data.get_pached_samples()
test_X = loger(test_X)
_X_test = test_data.scaling_data_and_flatten(test_X, scaler)

In [ ]:
test_dataset = RBMDataset(_X_test)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
test_features = np.zeros((len(_X_test), HIDDEN_UNITS))
for i, batch in enumerate(test_dataloader):
    #batch = batch.view(len(batch), VISIBLE_UNITS)  # flatten input data

    if CUDA:
        batch = batch.cuda()

    test_features[i*BATCH_SIZE:i*BATCH_SIZE+len(batch)] = rbm.sample_hidden(batch).cpu().numpy()

ОТРИСОВКА


In [ ]:
pca = PCA(n_components=2)
features_PCA=pca.fit_transform(test_features)
df_pca = pd.DataFrame()
df_pca['Parameter_1'] = features_PCA[:,[0]].reshape(-1)
df_pca['Parameter_2'] = features_PCA[:,[1]].reshape(-1)

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0, algorithm = "full")
clusters_labels=kmeans.fit_predict(test_features)
df_pca["Real_labels"] = test_data.get_labels()
df_pca["Cluster_label"]=clusters_labels

In [ ]:
df_pca[:2]

,Parameter_1,Parameter_2,Real_labels,Cluster_label
0,0.866930,0.604383,1.0,1
1,1.364407,-0.396831,1.0,1


In [ ]:
fig = px.scatter(df_pca, x="Parameter_1", y="Parameter_2", color="Real_labels")
fig.show()

In [ ]:
fig = px.scatter(df_pca, x="Parameter_1", y="Parameter_2", color="Cluster_label", title="Кластериация c преобразованием log > корень 3-степени")
fig.show()

In [ ]:
test_frame = test_data.return_OHLV()

In [ ]:
up_indexes =np.where(clusters_labels==1)
up_indexes=np.array(up_indexes[0])
down_indexes = np.where(clusters_labels==0)
down_indexes = np.array(down_indexes[0])

In [ ]:
import plotly.graph_objects as go



fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(len(test_frame))], y=test_frame["Close"],
                    mode='lines',
                    name='close'))

fig.add_trace(go.Bar(
    x=up_indexes,
    y=[test_frame["Close"].max() for i in range(len(up_indexes))],
    name='up',
    marker_color='green', opacity=0.3
))
fig.add_trace(go.Bar(
    x=down_indexes,
    y=[test_frame["Close"].max() for i in range(len(down_indexes))],
    name='down',
    marker_color='red',opacity=0.3
))
fig.update_layout(
    title="Размеченное движение цены на форварде с преобразованием log > корень 3-степени")
fig.update_yaxes(range=[test_frame["Close"].min(), test_frame["Close"].max()])

fig.show()